In [1]:
! pip install wandb -qU

     |████████████████████████████████| 1.8 MB 37.9 MB/s 
     |████████████████████████████████| 181 kB 76.4 MB/s 
     |████████████████████████████████| 147 kB 74.6 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 


In [2]:
import time
import tensorflow as tf
from tensorflow.keras.layers import Flatten,Dense,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import wandb
from wandb.keras import WandbCallback



import numpy as np
import cv2
import matplotlib.pyplot as plt

In [3]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
wandb.init(project="Face Mask Detection", entity="gharabadiyan")

wandb: Currently logged in as: gharabadiyan. Use `wandb login --relogin` to force relogin


In [5]:
train_dataset_path = '/content/drive/MyDrive/Face Mask Dataset/Train'
valid_dataset_path = '/content/drive/MyDrive/Face Mask Dataset/Validation'

width = height = 224
batch_size = 32

idg = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    brightness_range = (0.6,1.2),
    zoom_range = 0.1,
    shear_range = 0.3,
    rotation_range = 15,
)

train_data = idg.flow_from_directory(
    train_dataset_path,
    target_size = (width,height),
    class_mode = "categorical",
    batch_size = batch_size,
    subset = "training"
)
val_data = idg.flow_from_directory(
    valid_dataset_path,
    target_size = (width,height),
    class_mode = "categorical",
    batch_size = batch_size,
)



Found 10012 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [6]:
base_model = tf.keras.applications.MobileNetV2(
    include_top=False,
    weights = 'imagenet',
    input_shape=(width,height,3),
    pooling = 'avg',
)

model = tf.keras.models.Sequential([
        base_model,
        Dropout(0.3), 
        Dense(2,activation='softmax')
])

9420800/9406464 [==============================] - 0s 0us/step


In [7]:
for layer in base_model.layers[:-2]:  #freeze layer
  layer.trainable = False

In [8]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

In [9]:
model.fit(train_data,validation_data=val_data,epochs=10,callbacks=[WandbCallback()])

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
Epoch 1/10
313/313 [==============================] - 2468s 8s/step - loss: 0.1124 - accuracy: 0.9572 - val_loss: 0.0315 - val_accuracy: 0.9937 - _timestamp: 1658396218.0000 - _runtime: 2503.0000
Epoch 2/10
313/313 [==============================] - 145s 464ms/step - loss: 0.0469 - accuracy: 0.9837 - val_loss: 0.0263 - val_accuracy: 0.9887 - _timestamp: 1658396363.0000 - _runtime: 2648.0000
Epoch 3/10
313/313 [==============================] - 145s 465ms/step - loss: 0.0382 - accuracy: 0.9881 - val_loss: 0.0241 - val_accuracy: 0.9937 - _timestamp: 1658396509.0000 - _runtime: 2794.0000
Epoch 4/10
313/313 [==============================] - 145s 464ms/step - loss: 0.0362 - accuracy: 0.9871 - val_loss: 0.0206 - val_accuracy: 0.9912 - _timestamp: 1658396654.0000 - _runtime: 2939.0000
Epoch 5/10
313/313 [==============================] - 145s 462ms/step - loss: 0.0342 - accuracy: 0.9885 - val_loss: 0.016

In [10]:
model.save('Face_Mask.h5')

In [11]:
test_dataset_path = '/content/drive/MyDrive/Face Mask Dataset/Test'
width=height = 224
batch_size = 32
idg = ImageDataGenerator(
    rescale = 1./255,
)


test_data = idg.flow_from_directory(
    test_dataset_path,
    target_size = (width,height),
    class_mode = "categorical",
    batch_size = batch_size,
)

Found 992 images belonging to 2 classes.


In [12]:
model.evaluate(test_data)

31/31 [==============================] - 199s 7s/step - loss: 0.0223 - accuracy: 0.9940


[0.022283410653471947, 0.9939516186714172]